# PLaMo-13BのJCommonsenseQAによる評価

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
N_SAMPLES = 10
# N_SAMPLES = None
# full setでの評価には上記のコメントアウトを外す

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "pfnet/plamo-13b",
    trust_remote_code=True,
)
model = AutoModelForCausalLM.from_pretrained(
    "pfnet/plamo-13b",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
model.config

### データセットのダウンロード

In [ ]:
dataset = load_dataset("leemeng/jcommonsenseqa-v1.1", split="validation")

if N_SAMPLES is not None:
    shuffled_dataset = dataset.shuffle(seed=42)
    dataset = shuffled_dataset.select(range(N_SAMPLES))

dataset

### 質問-回答例のサンプリング

コンテクストとして与える質問と模範回答の例を作成する。モデルの評価にはvalidation splitを利用しているのでここではtrain splitからサンプリングする。

In [ ]:
train_dataset = load_dataset("leemeng/jcommonsenseqa-v1.1", split="train")

In [ ]:
display(train_dataset[0])

In [ ]:
sample_prompt = """質問: 主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？
choice0: 世界
choice1: 写真集
choice2: 絵本
choice3: 論文
choice4: 図鑑
回答: 絵本"""

### テキストの生成

In [ ]:
for i, item in tqdm(enumerate(dataset), total=dataset.num_rows):
    text = f"""質問: {item["question"]}\nchoice0: {item["choice0"]}\nchoice1: {item["choice1"]}\nchoice2: {item["choice2"]}\nchoice3: {item["choice3"]}\nchoice4: {item["choice4"]}\n解答: """
    prompt_text = f"### 例 ###\n{sample_prompt}\n\n{text}"
    print(prompt_text)
    break

In [ ]:
answers = []

for i, item in tqdm(enumerate(dataset), total=dataset.num_rows):
    text = f"""質問: {item["question"]}\nchoice0: {item["choice0"]}\nchoice1: {item["choice1"]}\nchoice2: {item["choice2"]}\nchoice3: {item["choice3"]}\nchoice4: {item["choice4"]}\n解答: """
    prompt_text = f"### 例 ###\n{sample_prompt}\n\n{text}"
    prompt = tokenizer(prompt_text, return_tensors="pt").input_ids
    prompt_len = len(prompt[0])
    prompt = prompt.to(model.device)
    generated_tokens = model.generate(
        inputs=prompt,
        max_new_tokens=5,
        do_sample=False,
    )[0]
    generated_text = tokenizer.decode(generated_tokens[prompt_len:])
    answer = generated_text.split("\n")[0]
    answers.append(answer)

In [ ]:
correct_answers = []
for item in dataset:
    choices = [item[f"choice{i}"] for i in range(5)]
    label = item["label"]
    correct_answers.append(choices[label])

In [ ]:
qa_df = dataset.to_pandas()
qa_df["answer"] = answers
qa_df["correct_answer"] = correct_answers
tag = f"_sample{N_SAMPLES}" if N_SAMPLES is not None else ""
qa_df.to_csv(f"jcommonsense_plamo13b{tag}.csv")
qa_df

### 評価

In [ ]:
def norm_acc(df):
    df = df.copy()
    accs = []
    for label, df_ in df.groupby("label"):
        acc = df_.apply(lambda item: item["answer"] == item["correct_answer"], axis=1).mean()
        accs.append(acc)
    return np.mean(accs), accs

In [ ]:
acc, accs = norm_acc(qa_df)
print(f"norm_acc(1-shot): {100 * acc:.1f}%")
print([f"{i}: {acc * 100:.1f}%" for i, acc in enumerate(accs)])

## 誤答の例

In [ ]:
error_df = qa_df.query("answer != correct_answer")

for _, item in error_df.sample(min(100, len(error_df))).iterrows():
    print(
        f"""問題: {item["question"]}
選択肢: {item["choice0"]}, {item["choice1"]}, {item["choice2"]}, {item["choice3"]}, {item["choice4"]}
正答: {item["correct_answer"]}
モデルの回答: {item["answer"]}
"""
    )